<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____
<a id='home'></a>

# Introduction to Optimization


In [3]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>


Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.

In [4]:
from pulp import LpMaximize,LpProblem,LpVariable,LpConstraint, value
from pulp import COIN_CMD
from pulp import LpConstraintGE as GE
from pulp import LpConstraintLE as LE


model = LpProblem(name='refinery-problem', sense=LpMaximize)

gas = LpVariable(name="Gas", lowBound=0,cat='Continuous')
oil = LpVariable(name="Oil", lowBound=0,cat='Continuous')

obj_func = 1.9*gas + 1.5*oil

C1= LpConstraint(name='Gas Constraint', 
                 e= 1*gas - 2*oil, sense=GE, rhs=0)
C2= LpConstraint(name='Oil Constraint',
                 e= 1*oil, sense=GE, rhs=3000000)
C3= LpConstraint(name='Demand Constraint',
                 e= 1*gas, sense=LE, rhs=6400000)


# build model with info and solve the model

model += obj_func
model += C1
model += C2
model += C3

solver = COIN_CMD(msg=False)
result=model.solve(solver)

In [5]:
print ("Optimal Result:")
for variable in model.variables():
    print (variable.name, "=", variable.varValue)

Optimal Result:
Gas = 6400000.0
Oil = 3200000.0


In [6]:
print ("Total net max profit:")
print (value(model.objective))

Total net max profit:
16960000.0


In [7]:
print ("RHS-LHS")
for name, constraint in model.constraints.items():
    print(name + ':' + str(constraint.value()))

RHS-LHS
Gas_Constraint:0.0
Oil_Constraint:200000.0
Demand_Constraint:0.0


In [8]:
import pandas as pd

o = [{'name':name,'shadow price':c.pi,'slack': c.slack} for name, c in model.constraints.items()]

print(pd.DataFrame(o))

                name  shadow price     slack
0     Gas_Constraint         -0.75      -0.0
1     Oil_Constraint         -0.00 -200000.0
2  Demand_Constraint          2.65      -0.0


In [15]:
%%html


<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [10]:
from pulp import LpMinimize

model = LpProblem(name='diet-problem', sense=LpMinimize)

V_V = LpVariable(name="Vega Vita", lowBound=0,cat='Integer')
H_H = LpVariable(name="Happy Health", lowBound=0,cat='Integer')

obj_func = 0.2*V_V + 0.3*H_H

C1=LpConstraint(e=20*V_V + 30*H_H,sense=GE,name='Vitamin C',rhs=60)
C2=LpConstraint(e=500*V_V + 250*H_H,sense=GE,name='Calcium',rhs=1000)
C3=LpConstraint(e=9*V_V + 2*H_H,sense=GE,name='Iron',rhs=18)
C4=LpConstraint(e=2*V_V + 10*H_H,sense=GE,name='Niacin',rhs=20)
C5=LpConstraint(e=60*V_V + 90*H_H,sense=GE,name='Magnesium',rhs=360)


model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

solver = COIN_CMD(msg=False)
result=model.solve(solver)

In [11]:
print ("Optimal Result:")
for variable in model.variables():
    print (variable.name, "=", variable.varValue)

Optimal Result:
Happy_Health = 2.0
Vega_Vita = 3.0


In [12]:
print ("Total min expenses:")
print (value(model.objective))

Total min expenses:
1.2000000000000002


In [13]:
print ("RHS-LHS")
for name, constraint in model.constraints.items():
    print(name + ':' + str(constraint.value()))

RHS-LHS
Vitamin_C:60.0
Calcium:1000.0
Iron:13.0
Niacin:6.0
Magnesium:0.0


In [14]:
import pandas as pd

o = [{'name':name,'shadow price':c.pi,'slack': c.slack} for name, c in model.constraints.items()]

print(pd.DataFrame(o))

        name  shadow price   slack
0  Vitamin_C           0.0   -60.0
1    Calcium           0.0 -1000.0
2       Iron           0.0   -13.0
3     Niacin           0.0    -6.0
4  Magnesium           0.0    -0.0
